# Import the Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
# Numpy cries because it is an old code so we monkey patch it
np.int = int
np.bool = bool
np.float = float

# Import the Dataset

In [2]:
URM_all_dataframe = pd.read_csv('data_train.csv', 
                                sep=",", 
                                header= 0, 
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

### Building the sparse COO matrix using URM

In [3]:
import scipy.sparse as sps
# Build the COO sparse matrix associated with the URM
URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values))) ## .values --> numpy array, df[..] --> pd series

### Split the data in train and validation set

In [4]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2584 (19.8%) Users that have less than 1 test interactions


# Implement ItemBasedCF

In [6]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [7]:
recommender = ItemKNNCFRecommender(URM_train)
# Item-Based CF {'similarity': 'asymmetric', 'topK': 5, 'shrink': 18, 'asymmetric_alpha': 0.18548786443872095}


ItemKNNCFRecommender: URM Detected 606 ( 4.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 232 ( 1.0%) items with no interactions.


### The coefficient gamma represents how much in proportion we want to use TopPop()

In [8]:
recommender.fit(gamma = 1e-6, similarity = 'asymmetric', topK = 5, shrink = 18, asymmetric_alpha = 0.18548786443872095)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 1346.93 column/sec. Elapsed time 16.59 sec


In [13]:
result_df, _ = evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 10441 (100.0%) in 16.69 sec. Users per second: 626


In [14]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.091735,0.155873,0.130085,0.047465,0.080013,0.263822,0.144237,0.107595,0.5034,0.348844,...,0.801612,0.403532,0.801612,0.089589,10.592852,0.997586,0.258556,0.816077,1.77934,0.302827


In [46]:
def write_predictions(recommender_object, at=10):
    prediction_df = pd.read_csv('data_target_users_test.csv', sep= ",",
                                header=0, 
                                dtype={0:int},
                                engine='python')
    users = np.array(prediction_df['user_id'])
    scores = recommender_object._compute_item_score(users)
    recommendations = np.array([' '.join(map(str, np.argsort(-scores[i])[:at])) for i in range(len(users))])
    prediction_df['item_list'] = recommendations
    print(prediction_df.head(10))
    prediction_df.to_csv('submission.csv',index=False)

write_predictions(recommender)

   user_id                                         item_list
0        1           36 161 694 592 699 354 618 342 1481 403
1        2             47 1095 17 27 42 28 1227 1522 1372 50
2        3       59 857 3152 648 750 956 2925 1895 2172 1097
3        4                 17 249 27 42 119 28 47 50 252 299
4        5             77 131 170 66 60 1570 3063 5138 270 4
5        6                886 351 9 35 168 692 330 546 14 16
6        8        443 722 769 210 451 1155 480 121 1749 1996
7        9              2821 12741 10108 8133 6415 2 4 1 3 7
8       10  1446 1316 1816 1668 1674 1411 1382 809 2565 2145
9       11                 955 346 627 681 67 41 99 25 31 40
